In [1]:
!pip install sparse==0.7.0
!pip install efficientnet_pytorch
!pip install scipy==1.4.1
!pip install torchio

     |████████████████████████████████| 48 kB 335 kB/s eta 0:00:011
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.0-py3-none-any.whl size=16035 sha256=521566f1f40d3c0b20760b21de51e9dbfd027a5f545ec30c0b7f3f24c44e8783
  Stored in directory: /root/.cache/pip/wheels/b7/cc/0d/41d384b0071c6f46e542aded5f8571700ace4f1eb3f1591c29
Successfully built efficientnet-pytorch
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
     |████████████████████████████████| 108 kB 581 kB/s eta 0:00:01
  Created wheel for Python-Deprecated: filename=Python_Deprecated-1.1.0-py2.py3-none-any.whl size=3069 sha256=80284587e44dadfa89b4f9ec81db290625f7ff716045816cdae0e007767480b4
  Stored in directory: /root/.cache/pip/wheels/e2/6a/3e

In [2]:
import platform
import os
from collections import namedtuple, defaultdict

import time

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import Dataset, DataLoader
import torchio


import numpy as np
import pandas as pd
import sparse

from efficientnet_pytorch import EfficientNet

from sklearn.model_selection import train_test_split


RUNNING_IN_KAGGLE = 'linux' in platform.platform().lower()
IMAGE_PATH = "../input/osic-pulmonary-fibrosis-progression/" if RUNNING_IN_KAGGLE else '/Users/Macbook/datasets/KaggleOSICPulmonaryFibrosisProgression'
PROCESSED_PATH = '../input/osic-processed/' if RUNNING_IN_KAGGLE else '/Users/Macbook/datasets/processed-data/'  # TODO: fix this line

dtype = torch.float32
USE_GPU = True
if USE_GPU and torch.cuda.is_available():
    device = 'cuda:0'
else:
    device = 'cpu'
device = torch.device(device)

If you use TorchIO for your research, please cite the following paper:
Pérez-García et al., TorchIO: a Python library for efficient loading,
preprocessing, augmentation and patch-based sampling of medical images
in deep learning. Credits instructions: https://torchio.readthedocs.io/#credits



In [3]:
root = f'{PROCESSED_PATH}/train'
patients = list(sorted(os.listdir(root)))
stats = defaultdict(list)
for patient in patients:
    base_path = os.path.join(root, patient)
    meta = np.load(os.path.join(base_path, 'meta.npy'), allow_pickle=True).tolist() 
    meta_processed = dict()
    for key, values in meta.items():
        if key in {'SliceLocation', 'InstanceNumber'}:
            continue
        else:
            unique_values, values_cnt = np.unique(values, return_counts=True, axis=0)
            most_frequent = sorted(zip(unique_values, values_cnt), key=lambda x: x[1])[-1][0]
            most_frequent = np.array(most_frequent).reshape(-1)
            if key in {
                'SliceThickness', 'TableHeight', 'WindowCenter', 'WindowWidth'
            }:
                meta_processed[key] = most_frequent[0]
            elif key == 'PixelSpacing':
                if len(most_frequent) == 1:
                    meta_processed['PixelSpacingX'], meta_processed['PixelSpacingY'] = (
                        most_frequent[0], most_frequent[0]
                    )
                else:
                    meta_processed['PixelSpacingX'], meta_processed['PixelSpacingY'] = (
                        most_frequent[0], most_frequent[1]
                    )
            elif key == 'PatientPosition':
                pass
            elif key == 'PositionReferenceIndicator':
                pass
    for key in meta_processed:
        stats[key] += [meta_processed[key]]

In [4]:
ct_stats = {}
for key, values in stats.items():
    values = np.array(values)
    stat = namedtuple('mean', 'std')
    stat.mean = values.mean()
    stat.std = values.std()
    ct_stats[key] = stat

In [59]:
for key in ct_stats:
    print(key, 'mean: ', ct_stats[key].mean, 'std: ', ct_stats[key].std)

PixelSpacingX mean:  2.7656187595460904 std:  2.3477744475149662
PixelSpacingY mean:  1.4237380549311638 std:  0.14756358563813035
SliceThickness mean:  1.256083 std:  0.99084175
TableHeight mean:  133.76607959659088 std:  58.598966674166675
WindowCenter mean:  -523.8579545454545 std:  192.05943458731193
WindowWidth mean:  -1241.5454545454545 std:  848.268562869134


In [ ]:
for key in meta_stats:
    print(key, 'mean: ', meta_stats[key].mean, 'std: ', meta_stats[key].std)

In [5]:
meta_stats = dict.fromkeys(['Weeks', 'FVC', 'Age'])
meta_df = pd.read_csv(f'{IMAGE_PATH}/train.csv')
for key in meta_stats:
    values = meta_df[key]
    stat = namedtuple('mean', 'std')
    stat.mean = values.mean()
    stat.std = values.std()
    meta_stats[key] = stat
stat = namedtuple('mean', 'std')
stat.mean = 971.4692260919278
stat.std = 117.84143467421829
meta_stats['Lungs'] = stat

In [6]:
class CTDataset(Dataset):
    _ReturnValue = namedtuple('ReturnValue', ['weeks', 'fvcs', 'features', 'masks', 'images'])

    def __init__(
            self, root, csv_path, train=True, test_size=0.25, random_state=42,
            padding_mode=None, padding_constant=None, transform=None
    ):
        assert test_size is not None
        assert padding_mode in {None, 'edge', 'mean', 'max_min', 'constant'}
        assert (padding_mode == 'constant' and padding_constant is not None) or (padding_mode != 'constant')

        self.root = root
        self.train = train
        self.transform = transform
        self.csv_path = csv_path
        self.test_size = test_size
        self.random_state = random_state
        self.padding_mode = padding_mode
        self.padding_constant = padding_constant

        if not os.path.exists(self.root):
            raise ValueError('Data is missing')

        self._patients = list(sorted(os.listdir(self.root)))

        if self.test_size == 0:
            self._train_patients, self._test_patients = self._patients, []
        else:
            self._train_patients, self._test_patients = train_test_split(
                self._patients, test_size=self.test_size, random_state=random_state
            )

        self._table_features = dict()
        table_data = pd.read_csv(self.csv_path)
        for patient in self._patients:
            patient_data = table_data[table_data.Patient == patient]

            all_weeks = patient_data.Weeks.tolist()
            all_fvcs = patient_data.FVC.tolist()

            all_weeks, all_fvcs = zip(*sorted(zip(all_weeks, all_fvcs), key=lambda x: x[0]))

            age = sorted(zip(*np.unique(patient_data.Age, return_counts=True)), key=lambda x: x[1])[-1][0]
            sex = sorted(zip(*np.unique(patient_data.Sex, return_counts=True)), key=lambda x: x[1])[-1][0]
            smoking_status = \
                sorted(zip(*np.unique(patient_data.SmokingStatus, return_counts=True)), key=lambda x: x[1])[-1][0]

            sex = [0] if sex == 'Female' else [1]
            smoking_status = (
                [1, 0] if smoking_status == 'Ex-smoker' else
                [0, 1] if smoking_status == 'Never smoked' else
                [0, 0]
            )
            
            age = (age - meta_stats['Age'].mean) / meta_stats['Age'].std # add normalization
            self._table_features[patient] = (
                all_weeks, all_fvcs, [age] + sex + smoking_status
            )

    def __getitem__(self, index):
        """
        Args:
            index (int): Index
        Returns:
            tuple: (image, target) where target is index of the target class.
        """
        patient = self._train_patients[index] if self.train else self._test_patients[index]
        base_path = os.path.join(self.root, patient)

        # noinspection PyTypeChecker
        meta = np.load(os.path.join(base_path, 'meta.npy'), allow_pickle=True).tolist()  # type: Dict[List]
        masks = sparse.load_npz(os.path.join(base_path, 'masks.npz'))
        images = np.load(os.path.join(base_path, 'images.npy'))

        meta_processed = dict()
        for key, values in meta.items():
            if key in {'SliceLocation', 'InstanceNumber'}:
                continue
            else:
                unique_values, values_cnt = np.unique(values, return_counts=True, axis=0)
                most_frequent = sorted(zip(unique_values, values_cnt), key=lambda x: x[1])[-1][0]
                most_frequent = np.array(most_frequent).reshape(-1)
                if key in {
                    'SliceThickness', 'TableHeight', 'WindowCenter', 'WindowWidth'
                }:
                    meta_processed[key] = most_frequent[0]
                elif key == 'PixelSpacing':
                    if len(most_frequent) == 1:
                        meta_processed['PixelSpacingX'], meta_processed['PixelSpacingY'] = (
                            most_frequent[0], most_frequent[0]
                        )
                    else:
                        meta_processed['PixelSpacingX'], meta_processed['PixelSpacingY'] = (
                            most_frequent[0], most_frequent[1]
                        )
                elif key == 'PatientPosition':
                    pass
                elif key == 'PositionReferenceIndicator':
                    pass
        
        # add normalization
        for key in meta_processed:
            meta_processed[key] = (meta_processed[key] - ct_stats[key].mean) / ct_stats[key].std
        
        all_weeks, all_fvcs, features = self._table_features[patient]
        features = [value for key, value in meta_processed.items()] + features

        if self.padding_mode is None:
            pass
        if self.padding_mode == 'edge':
            all_weeks, all_fvcs = [-13] + all_weeks + [133], [all_fvcs[0]] + all_fvcs + [all_fvcs[1]]
        if self.padding_mode == 'mean':
            all_weeks, all_fvcs = [-13] + all_weeks + [133], [np.mean(all_fvcs)] + all_fvcs + [np.mean(all_fvcs)]
        if self.padding_mode == 'max_min':
            all_weeks, all_fvcs = [-13] + all_weeks + [133], [np.max(all_fvcs)] + all_fvcs + [np.min(all_fvcs)]
        if self.padding_mode == 'constant':
            all_weeks, all_fvcs = [-13] + all_weeks + [133], [self.padding_constant] + all_fvcs + [
                self.padding_constant]
        
        images = torch.tensor(images[None, :, :, :], dtype=dtype)
        masks = torch.tensor(masks[None, :, :, :].todense(), dtype=dtype)
        
        if self.transform is not None:
            # noinspection PyTypeChecker
            subject = torchio.Subject(
                masks=torchio.ScalarImage(tensor=masks),
                images=torchio.ScalarImage(tensor=images)
            )
            transformed_subject = self.transform(subject)
            masks = transformed_subject.masks
            images = transformed_subject.images

        return CTDataset._ReturnValue(weeks=all_weeks, fvcs=all_fvcs, features=features, masks=masks, images=images)

    def __len__(self):
        return len(self._train_patients if self.train else self._test_patients)

    def __repr__(self):
        fmt_str = 'OSIC Pulmonary Fibrosis Progression Dataset ' + self.__class__.__name__ + '\n'
        fmt_str += '    Number of datapoints: {}\n'.format(self.__len__())
        tmp = 'train' if self.train is True else 'test'
        fmt_str += '    Split: {}\n'.format(tmp)
        fmt_str += '    Root Location: {}\n'.format(self.root)
        return fmt_str

In [7]:
transforms = torchio.transforms.Compose([
    torchio.transforms.RandomAffine(degrees=(10, 10),
                                    translation=(-10, -10), 
                                    isotropic=False, 
                                    default_pad_value='minimum', 
                                    image_interpolation='linear')])

In [8]:
train_dataset = CTDataset(
    f'{PROCESSED_PATH}/train',
    f'{IMAGE_PATH}/train.csv',
    train=True, test_size=0.25, random_state=42, transform=None
)

val_dataset = CTDataset(
    f'{PROCESSED_PATH}/train',
    f'{IMAGE_PATH}/train.csv',
    train=False, test_size=0.25, random_state=42, transform=None
)

In [9]:
len(train_dataset), len(val_dataset)

(132, 44)

In [10]:
# train_dataloader = DataLoader(train_dataset, batch_size=1, num_workers=3)
# val_dataloader = DataLoader(val_dataset, batch_size=1, num_workers=3)
# next(iter(train_dataloader))

In [11]:
def laplace_loss(y_true, y_pred, log_sigma):
    log_sigma = torch.clamp(log_sigma, -5, 5)
    losses = np.sqrt(2) * (y_true - y_pred).abs() / log_sigma.exp() + log_sigma + np.log(2) / 2
    return losses.mean()

In [38]:
def mse_loss(y_true, y_pred, log_sigma):
    losses = (y_true - y_pred)**2 + (log_sigma - np.log(70))**2
    return losses.mean()

In [44]:
class Net2D(nn.Module):
    
    def __init__(self):
        super(Net2D, self).__init__()
        
        self.efficient_net_1 = EfficientNet.from_pretrained('efficientnet-b0', in_channels=1)
        self.batch_norm = nn.BatchNorm2d(num_features=1280)
        self.efficient_net_2 = EfficientNet.from_pretrained('efficientnet-b0', in_channels=1280)

        self.fc_1 = nn.Linear(1280 + 12, 500)
        self.fc_2 = nn.Linear(500, 250)
        self.fc_3 = nn.Linear(250, 5)
        
        self._initialize_weights()
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, torch.nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, np.sqrt(2. / n))
            elif isinstance(m, torch.nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)
    
    def forward(self, X, meta_X):
        """
        X: tensor (s, h, w): s - slices
        meta_X: tensor (n, 12)
        """
        
        X = X[::3, :, :]
        X = X.unsqueeze(1)
        X = self.efficient_net_1.extract_features(X)
        
        X = X.view(X.shape[0], 1280, 64) # shape (s, 1280, 64)
        
        X = X.unsqueeze(0) # shape (1, s, 1280, 64)
        X = X.transpose(1, 2) # shape (1,  1280, s, 64)
        
        X = self.batch_norm(X)
        
        X = self.efficient_net_2.extract_features(X) # shape (1, 1280, 2, 2)
        X = torch.mean(X, dim=(2,3)) # shape (1, 1280)
#       X = X.view(1, 5120)
        
        X = torch.cat([X.repeat(meta_X.shape[0], 1), meta_X], dim=1) 
        
        X = F.relu(self.fc_1(X))
        X = F.relu(self.fc_2(X))
        y = self.fc_3(X)
        
        return y

In [58]:
X = torch.rand(1,1280, 64, 64)
X = nn.Conv2d(1280, 640, 3, 3)(X)
X = F.relu(X)
X = nn.Conv2d(640, 320, 3, 3)(X)
X = F.relu(X)
X = nn.Conv2d(320, 320, 3, 3)(X)
X = F.relu(X)
X.shape
X.view(1,-1).shape

torch.Size([1, 1280])

In [ ]:
class Net2D_2(nn.Module):
    
    def __init__(self):
        super(Net2D, self).__init__()
        
        self.efficient_net_1 = EfficientNet.from_pretrained('efficientnet-b0', in_channels=1)
        self.batch_norm_1 = nn.BatchNorm2d(num_features=1280)
        self.conv_1 = nn.Conv2d(1280, 640, 3, 3)
        self.batch_norm_2 = nn.BatchNorm2d(num_features=1280)
        self.conv_2 = nn.Conv2d(640, 320, 3, 3)
        self.batch_norm_1 = nn.BatchNorm2d(num_features=1280)
        self.conv_3 = nn.Conv2d(320, 320, 3, 3)
        

        self.fc_1 = nn.Linear(1280 + 12, 500)
        self.fc_2 = nn.Linear(500, 250)
        self.fc_3 = nn.Linear(250, 5)
        
        self._initialize_weights()
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, torch.nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, np.sqrt(2. / n))
            elif isinstance(m, torch.nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)
    
    def forward(self, X, meta_X):
        """
        X: tensor (s, h, w): s - slices
        meta_X: tensor (n, 12)
        """
        
        X = X[::3, :, :]
        X = X.unsqueeze(1)
        X = self.efficient_net_1.extract_features(X)
        
        X = X.view(X.shape[0], 1280, 64) # shape (s, 1280, 64)
        
        X = X.unsqueeze(0) # shape (1, s, 1280, 64)
        X = X.transpose(1, 2) # shape (1,  1280, s, 64)
        
        X = self.batch_norm(X)
        
        X = nn.Conv2d(X)
        X = F.relu(X)
        X = nn.Conv2d(640, 320, 3, 3)(X)
        X = F.relu(X)
        X = nn.Conv2d(320, 320, 3, 3)(X)
        X = F.relu(X)
        X.view(1,-1)

        X = self.efficient_net_2.extract_features(X) # shape (1, 1280, 2, 2)
        X = torch.mean(X, dim=(2,3)) # shape (1, 1280)
#       X = X.view(1, 5120)
        
        X = torch.cat([X.repeat(meta_X.shape[0], 1), meta_X], dim=1) 
        
        X = F.relu(self.fc_1(X))
        X = F.relu(self.fc_2(X))
        y = self.fc_3(X)
        
        return y

In [13]:
def polynom(coords, coefs):
    # coords shape (n, )
    # coefs shape (4, )
    
    poly_coords = torch.empty((coords.shape[0], 4), dtype=dtype, device=device)
    poly_coords[:, 3] = 1
    poly_coords[:, 2] = coords
    poly_coords[:, 1] = coords**2
    poly_coords[:, 0] = coords**3
    return (poly_coords * coefs.unsqueeze(0)).sum(dim=1)

In [32]:
def train_model(model, optimizer, loss_func, train_dataset, val_dataset, epochs, scheduler=None):

    def get_lr(optimizer):
        for param_group in optimizer.param_groups:
            return param_group['lr']

    val_loss_min = np.inf
    for epoch in range(epochs):
        running_loss = 0.0
        
#         for iter_num, data in enumerate(train_dataset):
        data = train_dataset[0]
        iter_num = 0
        model.train()
        optimizer.zero_grad()

        #prepare lungs
        masks = data.masks 
        images = data.images

        lungs = -1000 * (1.0 - masks) + masks * images
        lungs = (lungs - meta_stats['Lungs'].mean) / meta_stats['Lungs'].std
        
        lungs = lungs.squeeze(0)
        X = lungs.to(device)

        # prepare features
        weeks = torch.tensor(data.weeks, dtype=dtype, device=device)
        fvcs = torch.tensor(data.fvcs, dtype=dtype, device=device)
        features = torch.tensor(data.features, dtype=dtype, device=device)
        num_weeks = len(weeks)
        meta_weeks = (weeks - meta_stats['Weeks'].mean) / meta_stats['Weeks'].std
        meta_fvcs = (fvcs - meta_stats['FVC'].mean) / meta_stats['FVC'].std
        
        meta_X = torch.cat([meta_weeks.unsqueeze(1), meta_fvcs.unsqueeze(1), 
                               features.unsqueeze(0).repeat(num_weeks, 1)], dim=1)
        
        preds = model(X, meta_X) # shape (num_weeks, 5)


        coefs = preds[:, 0:4]
        log_sigma = preds[:, 4]

        loss = 0.0
        lap_loss = 0.0
        for week in range(num_weeks):
            fvcs_pred = polynom(weeks, coefs[week])
            loss += loss_func(fvcs, fvcs_pred, log_sigma[week])
            lap_loss += laplace_loss(fvcs.detach(), fvcs_pred.detach(), log_sigma[week].detach())

        loss /= num_weeks
        lap_loss /= num_weeks
        loss.backward() 
        optimizer.step()
        running_loss += loss.item()

        print("Epoch: {} ".format(epoch + 1),
              "Iteration: {} ".format(iter_num),
              "lr: {:.6f} ".format(get_lr(optimizer)),
              "Loss: {:.6f}.".format(loss.item()),
              'Laplace loss: {:.6f}.'.format(lap_loss.item()),
               "Sigma: {:.6f}".format(log_sigma.mean().item()))

    running_loss /= len(train_dataset)  
        
#         val_loss = eval_model(model, loss_func, val_dataset)
    
        
#         print("Epoch: {}/{}...".format(epoch + 1, n_epochs),
#               "lr: {:.6f}...".format(get_lr(optimizer)),
#               "Loss: {:.6f}...".format(running_loss.item()),
#               "Val Loss: {:.6f}".format(val_loss))
#         print('------------------------------')

        
#         if val_loss <= val_loss_min:
#             torch.save(model.state_dict(), f'./state_dict{epoch}.pt')
#             print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(val_loss_min, val_loss))
#             val_loss_min = val_loss
        
#         if scheduler is not None:
#             scheduler.step()

In [45]:
model = Net2D().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-2)

Loaded pretrained weights for efficientnet-b0
Loaded pretrained weights for efficientnet-b0


In [46]:
train_model(model, optimizer, mse_loss, train_dataset, val_dataset, epochs=100, scheduler=None)

torch.Size([1, 1280, 64, 64])
Epoch: 1  Iteration: 0  lr: 0.010000  Loss: 4082568.500000. Laplace loss: 2824.787109. Sigma: 0.000597
torch.Size([1, 1280, 64, 64])
Epoch: 2  Iteration: 0  lr: 0.010000  Loss: 935636238336.000000. Laplace loss: 57986.050781. Sigma: 2.645709
torch.Size([1, 1280, 64, 64])
Epoch: 3  Iteration: 0  lr: 0.010000  Loss: 100736784.000000. Laplace loss: 4838.610840. Sigma: 0.495149
torch.Size([1, 1280, 64, 64])
Epoch: 4  Iteration: 0  lr: 0.010000  Loss: 38320384.000000. Laplace loss: 6642.150879. Sigma: 0.054521
torch.Size([1, 1280, 64, 64])


KeyboardInterrupt: 

In [29]:
for param_group in optimizer.param_groups:
    param_group['lr'] /= 5

In [ ]:
# def eval_model(model, loss_func, val_dataset):
#     running_loss = 0.0
#     model.eval()    
#     for i, data in enumerate(val_dataset):
#         #prepare lungs
#         masks = data.masks 
#         images = data.images

#         lungs = -1000 * (1.0 - masks) + masks * images
    
#         X = transforms(lungs).to(device)

#         # prepare features
#         weeks = torch.tensor(data.all_weeks).to(device)
#         fvcs = torch.tensor(data.all_fvcs).to(device)
#         features = torch.tensor(data.features).to(device)

#         num_weeks = len(weeks)
#         meta_X = torch.concat([weeks.unsqueeze(1), fvcs.unsqueeze(1), 
#                                features.unsqueeze(0).repeat(num_weeks, 1)], dim=1)

#         preds = model(X, meta_X) # shape (num_weeks, 5)

#         coefs = pred[: 0:4]
#         log_sigma = pred[:, 5]

#         loss = 0
#         for i in range(num_weeks):
#             fwc_pred = polynom(weeks, coefs[i])
#             loss += loss_func(fwc, fwc_pred, log_sigma[i])

#         loss /= num_weeks
#         running_loss += loss.item()
    
#     return running_loss / len(val_dataset)